# to_video Examples

Export calcium imaging data to video files for presentations and sharing.

Output images/videos saved to `docs/_images/to_video/` for documentation.

In [ ]:
from pathlib import Path
import numpy as np
import mbo_utilities as mbo
import matplotlib.pyplot as plt

RAW_PATH = Path(r"D:/demo/raw")
SAVE_PATH = Path("../_images/to_video")
SAVE_PATH.mkdir(parents=True, exist_ok=True)

print(f"Output directory: {SAVE_PATH.resolve()}")

## Load Demo Data

Load raw ScanImage TIFFs with `mbo.imread()`:

In [ ]:
arr = mbo.imread(RAW_PATH)

print(f"Type: {type(arr).__name__}")
print(f"Shape (T, Z, Y, X): {arr.shape}")
print(f"Dtype: {arr.dtype}")
print(f"Planes: {arr.num_planes}, Frames: {arr.num_frames}")

## Example 1: Basic Video Export

Export a single z-plane to video with default settings:

In [ ]:
# basic export - plane 7, first 200 frames
plane_data = arr[:200, 7, :, :]

output = mbo.to_video(
    plane_data,
    SAVE_PATH / "example_basic.mp4",
    fps=30,
)
print(f"Created: {output} ({output.stat().st_size / 1024:.1f} KB)")

In [ ]:
# save a sample frame for documentation
frame = np.asarray(arr[100, 7, :, :])
vmin, vmax = np.percentile(frame, [1, 99.5])

fig, ax = plt.subplots(figsize=(6, 5))
ax.imshow(frame, cmap='gray', vmin=vmin, vmax=vmax)
ax.set_title(f'Sample Frame (plane 7, t=100)', fontsize=11)
ax.axis('off')
plt.tight_layout()
plt.savefig(SAVE_PATH / "frame_raw.png", dpi=150, bbox_inches='tight', facecolor='white')
plt.show()
print(f"Saved: frame_raw.png")

## Example 2: Speed Factor (10x Faster)

Use `speed_factor` to create fast previews for checking cell stability:

In [ ]:
# 10x speed preview - great for quick stability check
output = mbo.to_video(
    arr[:500, 7, :, :],
    SAVE_PATH / "example_10x_speed.mp4",
    fps=30,
    speed_factor=10,
)
print(f"Created: {output} ({output.stat().st_size / 1024:.1f} KB)")
print(f"500 frames at 10x speed = {500 / (30 * 10):.1f}s video")

## Example 3: Enhanced Quality

Use temporal smoothing and gamma correction for cleaner videos:

In [ ]:
output = mbo.to_video(
    arr[:300, 7, :, :],
    SAVE_PATH / "example_enhanced.mp4",
    fps=30,
    speed_factor=5,
    temporal_smooth=3,  # 3-frame rolling average reduces flicker
    gamma=0.8,          # brighten midtones
    quality=10,         # highest quality
)
print(f"Created: {output}")

In [ ]:
# show gamma comparison
frame = np.asarray(arr[100, 7, :, :]).astype(np.float32)
vmin, vmax = np.percentile(frame, [1, 99.5])
frame_norm = np.clip((frame - vmin) / (vmax - vmin), 0, 1)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(frame_norm, cmap='gray')
axes[0].set_title('Gamma 1.0 (default)', fontsize=11)

axes[1].imshow(np.power(frame_norm, 0.7), cmap='gray')
axes[1].set_title('Gamma 0.7 (brighter)', fontsize=11)

axes[2].imshow(np.power(frame_norm, 1.3), cmap='gray')
axes[2].set_title('Gamma 1.3 (darker)', fontsize=11)

for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.savefig(SAVE_PATH / "gamma_comparison.png", dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

## Example 4: With Colormap

In [ ]:
output = mbo.to_video(
    arr[:200, 7, :, :],
    SAVE_PATH / "example_viridis.mp4",
    fps=30,
    speed_factor=5,
    cmap="viridis",
    gamma=0.8,
)
print(f"Created: {output}")

In [ ]:
# save viridis frame for docs
frame = np.asarray(arr[100, 7, :, :]).astype(np.float32)
vmin, vmax = np.percentile(frame, [1, 99.5])
frame_norm = np.clip((frame - vmin) / (vmax - vmin), 0, 1)
frame_gamma = np.power(frame_norm, 0.8)

fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(frame_gamma, cmap='viridis')
ax.set_title('Viridis Colormap (gamma=0.8)', fontsize=11)
ax.axis('off')
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig(SAVE_PATH / "frame_viridis.png", dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

## Example 5: 4D Data (Select Z-Plane)

For 4D arrays `(T, Z, Y, X)`, use `plane=` to select which z-plane to export:

In [ ]:
# export from full 4D array, selecting plane 7
output = mbo.to_video(
    arr[:200],  # full 4D: (200, 14, H, W)
    SAVE_PATH / "example_4d_plane7.mp4",
    fps=30,
    speed_factor=5,
    plane=7,  # select z-plane 7 (0-indexed)
    gamma=0.8,
)
print(f"Created: {output}")

## Example 6: Processing → Video

Load data, apply processing, convert to numpy, then export:

In [ ]:
# load and process
raw = np.asarray(arr[:300, 7, :, :])  # materialize to numpy
print(f"Raw shape: {raw.shape}, dtype: {raw.dtype}")

# apply processing: temporal mean subtraction (baseline correction)
baseline = raw.mean(axis=0, keepdims=True)
processed = raw - baseline + baseline.mean()

print(f"Processed shape: {processed.shape}")

In [ ]:
# export processed numpy array to video
output = mbo.to_video(
    processed,
    SAVE_PATH / "example_processed.mp4",
    fps=30,
    speed_factor=5,
    temporal_smooth=3,
    gamma=0.8,
    quality=10,
)
print(f"Created: {output}")

In [ ]:
# show raw vs processed
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

vmin, vmax = np.percentile(raw[100], [1, 99.5])
axes[0].imshow(raw[100], cmap='gray', vmin=vmin, vmax=vmax)
axes[0].set_title('Raw', fontsize=11)

vmin2, vmax2 = np.percentile(processed[100], [1, 99.5])
axes[1].imshow(processed[100], cmap='gray', vmin=vmin2, vmax=vmax2)
axes[1].set_title('Baseline Subtracted', fontsize=11)

for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.savefig(SAVE_PATH / "raw_vs_processed.png", dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

## Summary: Generated Files

In [ ]:
print("Generated files:")
print("=" * 50)
total_size = 0
for f in sorted(SAVE_PATH.glob("*")):
    size_kb = f.stat().st_size / 1024
    total_size += size_kb
    ext = f.suffix
    print(f"  {f.name:<35} {size_kb:>8.1f} KB")
print("=" * 50)
print(f"  {'Total':<35} {total_size:>8.1f} KB")